### MULTIPLE RUNS: Import Libraries, READ SWMM INP file, Setup MongoDB Database to store results

In [2]:
import yaml
from swmm_objects_3 import *
from swmm_read_3 import *
from swmm_run_3 import *
from pymongo import MongoClient
from random import randint

# Constants:
squareFeetInOneAcre = 43560.0
# Specify important files and parameters for the location:
swmmInitialInpFileNameStr = "wingohocking_PWD_y12m08d29-y12m09d05.inp"  # initial SWMM .inp file
# treatmentRatio = 0.081   # cfs/imperv_acre calibrated for Wingohocking T-14 outfall 7/1/12 - 6/30/13
lidDictFileNameStr = "LID_Dict.yaml"
# Calculate parameters for the watershed from the SWMM model INP file:
# watershedImpervAreaAcre = total_impervious_area(swmmInitialInpFileNameStr)
# Load YAML files 
f = open(lidDictFileNameStr,'r')   
lidDict = yaml.load(f)           # Read the LID ContribImpervArea and GreenedAcres
f.close()
# Compute a dictionary of contributing impervious ACRES for each LID 
contribImpervAreaAcre = {}
print("The following LID names are available:")
for lid in lidDict:
    print(lid)
    contribImpervAreaAcre[lid] = lidDict[lid]['ContribImpervArea'] / squareFeetInOneAcre  # acres
# Open the specified SWMM zeroLID INP file and obtain necessary data from it for multiple runs:    
swmmInitialInpStr = read_inp_file(swmmInitialInpFileNameStr)

# Specify MongoDB database server to use and login with credentials:    
mongoServer = 'mongodb://localhost:27017/'  # localhost
database = "fengwei"
client = MongoClient(mongoServer)
db = client[database]   # use or create the database
# db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database

collectionName = 'y18m05d21_RainBarrel_45_subcat_event_y12m08d29-y12m09d05'
#collectionName = 'y18m05d21_MorrisTT_45_subcat_event_y12m08d29-y12m09d05'
#collectionName = 'y18m05d21_wakefield_BR_RG_45_subcat_event_y12m08d29-y12m09d05'
thisRunCollection = db[collectionName]    # use or create the collection
contribImpervAreaAcre

The following LID names are available:
RainBarrel
MorrisTT
wakefield_BR_RG


{'MorrisTT': 0.6163911845730028,
 'RainBarrel': 0.1,
 'wakefield_BR_RG': 0.48230027548209364}

# Generate Zero LID Model Object & View the list of subcatchments where LID can be placed:

In [2]:
(section_names,sections) = read_inp(swmmInitialInpStr)
initialModel = swmm_model("InitialModel",section_names,sections)
lidShedsList = initialModel.getLidSubcatchmentNameList()
print("The following subcatchments can receive LIDs:")
print(lidShedsList)

The following subcatchments can receive LIDs:
['TT14-574_shed', 'T14-K3_shed', 'T14-B_shed', 'TT14-720_shed', 'TT14-646A_shed', 'TT14-552B_shed', 'T14-R6_shed', 'TT14-296B_shed', 'T14-E_shed', 'T14-J_shed', 'TT14-584_shed', 'T14-A2_shed', 'TT14-558_shed', 'TT14-502_shed', 'T14-P1_shed', 'T14-L2_shed', 'T14-A1_shed', 'T14-A3_shed', 'TT14-262A_shed', 'T14-P3_shed', 'TT14-620B_shed', 'T14-A4_shed', 'TT14-530_shed', 'TT14-524_shed', 'TT14-291D_shed', 'TT14-736_shed', 'TT14-708_shed', 'T14-D_shed', 'T14-M2_shed', 'T14-C1_shed', 'T14-R7_shed', 'T14-R1_shed', 'T14-C2_shed', 'T14-R5_shed', 'T14-M1_shed', 'T14-R2_shed', 'T14-P4_shed', 'TT14-706_shed', 'T14-G_shed', 'TT14-610C_shed', 'TT14-746_shed', 'TT14-740_shed', 'TT14-648A_shed', 'TT14-552A_shed', 'TT14-702_shed']


# Run a test simulation to check the software

In [3]:
# Run SWMM on the initial Zero LID model input:
swmmRptDict = run_swmm_and_return_results(swmmInitialInpStr)
#(section_names,sections) = read_inp(swmmInpStr)
#model = swmm_model("Model",section_names,sections)

0.0 min, 2.78 sec


# Print selected outputs from the test simulation

In [4]:
#print("Sewer Outflow Time Series:\n",swmmRptDict["outflow_seriesCFS"])

In [5]:
(section_names,sections) = read_inp(swmmInitialInpStr)
initialModel = swmm_model("InitialModel",section_names,sections)

lidShedsList = initialModel.getLidSubcatchmentNameList()


subcatAreaDict = {}            # Total area in each subcatchment (never changes)
subcatImpPctDict = {}          # ORIGINAL Impervious percentage in each subcatchment (changes when LID added)
subcatTreatableAreaDict = {}   # Original IMPERVIOUS area in zero LID SWMM file
#subcatMaxLID = {}              # Maximum number of each LID that can be installed in each subcatchment
subcatMaxGI = {}               # Maximum greened acres for each LID that can be installed in each subcatchment
# Now calculate the dictionaries defined above from data in the zero LID SWMM input file:

#lidShedsList = ['T14-B_shed','TT14-296B_shed']  # specify a subset of the LID subcatchments if desired
# Run all 45 subsheds:
for lidShed in lidShedsList:
    subcatArea = initialModel.getSubcatchmentArea(lidShed)
    subcatAreaDict[lidShed] = subcatArea
    pctImperv = initialModel.getSubcatchmentPctImperv(lidShed)
    subcatImpPctDict[lidShed] = pctImperv
    subcatTreatableAreaDict[lidShed] = subcatArea*pctImperv/100.0
#print(subcatTreatableAreaDict['TT14-574_shed'])    

maxTreatableFraction = 0.91
max_run = 500
lid = 'RainBarrel'
contribImpervArea = contribImpervAreaAcre[lid]  # acres

for lidShed in lidShedsList:
    print('In subshed ',lidShed,'using LID ',lid)
    #reinitialize the model with the original zero LID swmmInpStr:
    (section_names,sections) = read_inp(swmmInitialInpStr)  # Start with the initial NO LID SWMM input file
    model = swmm_model("Model",section_names,sections)   # This model will be modified below in the loops
    lidNumberBySubcatAndLidDict = {}

    subcatTreatableArea = maxTreatableFraction*subcatTreatableAreaDict[lidShed]   
    # set initial values before placing LIDs:
    subcatAreaTreated = 0.0
    numLID = 0
    numLIDIncrement = 1
    run = 0
    while run <= max_run:
        lidNumberByLidDict = {}
        subcatAreaTreated = subcatAreaTreated + numLID*contribImpervArea
        if subcatAreaTreated > subcatTreatableArea:
            break
        lidNumberByLidDict[lid] = numLID
        lidNumberBySubcatAndLidDict[lidShed] = lidNumberByLidDict
        model.lidChangeNumber(lidShed,lid,numLID)
        area = model.getLidArea(lidShed,lid)   # the lid footprint in the subcatchment 
        model.lidChangeArea(lidShed,lid,area,contribImpervAreaAcre[lid])  # must call to properly set FromImp parameter
        swmmInputFileStr=model.output()  # A modified SWMM input file containing the changed LID numbers
        totalGreenedAcreByLidDict = greened_acres_deployed(swmmInputFileStr,lidDict) # extract greened acres for each LID

        totalGreenedAcres = 0.0
        for l in totalGreenedAcreByLidDict:
            totalGreenedAcres += totalGreenedAcreByLidDict[l]
            
        # RUN SWMM and read it's output file creating a Report Dictionary
        swmmRptDict = run_swmm_and_return_results(swmmInputFileStr)
        
        # Append additional dictionaries to the Report Dictionary:
        swmmRptDict["lidDict"] = lidDict
        swmmRptDict["lidNumberBySubcatAndLidDict"] = lidNumberBySubcatAndLidDict
        
        # Insert the augmented report dictionary into the Mongo Database Collection specified above:
        docId = thisRunCollection.insert_one(swmmRptDict).inserted_id
        
        # Show user the progress through the runs:
        print("Run ",run," completed:")
        print("Stored record for a run with manual LID assignments with docId = ")
        print(docId)
        greenAcrePrintStr = 'Total Greened Acres = {:0.1f} with:'.format(totalGreenedAcres)
        lidGreenedAcreStr = ''
        for l in totalGreenedAcreByLidDict:
            lidGreenedAcreStr += " {:s} = {:0.1f},".format(l,totalGreenedAcreByLidDict[l])
        print(greenAcrePrintStr + lidGreenedAcreStr)
        numLID = numLID + numLIDIncrement
        run = run + 1
    print()


In subshed  TT14-574_shed using LID  RainBarrel
0.0 min, 2.73 sec
Run  0  completed:
Stored record for a run with manual LID assignments with docId = 
5b0364a6fa005d0e15f55848
Total Greened Acres = 0.0 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.83 sec
Run  1  completed:
Stored record for a run with manual LID assignments with docId = 
5b0364a9fa005d0e15f55849
Total Greened Acres = 0.1 with: RainBarrel = 0.1, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.85 sec
Run  2  completed:
Stored record for a run with manual LID assignments with docId = 
5b0364acfa005d0e15f5584a
Total Greened Acres = 0.2 with: RainBarrel = 0.2, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.77 sec
Run  3  completed:
Stored record for a run with manual LID assignments with docId = 
5b0364affa005d0e15f5584b
Total Greened Acres = 0.3 with: RainBarrel = 0.3, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.83 sec
Run  4  completed:
Stored record for a run with manual LID assignm

0.0 min, 2.66 sec
Run  1  completed:
Stored record for a run with manual LID assignments with docId = 
5b03650cfa005d0e15f5586e
Total Greened Acres = 0.1 with: RainBarrel = 0.1, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.57 sec
Run  2  completed:
Stored record for a run with manual LID assignments with docId = 
5b03650efa005d0e15f5586f
Total Greened Acres = 0.2 with: RainBarrel = 0.2, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.46 sec
Run  3  completed:
Stored record for a run with manual LID assignments with docId = 
5b036511fa005d0e15f55870
Total Greened Acres = 0.3 with: RainBarrel = 0.3, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.54 sec
Run  4  completed:
Stored record for a run with manual LID assignments with docId = 
5b036513fa005d0e15f55871
Total Greened Acres = 0.4 with: RainBarrel = 0.4, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.67 sec
Run  5  completed:
Stored record for a run with manual LID assignments with docId = 
5b036516fa005d0e15f55872
Tota

0.0 min, 3.61 sec
Run  39  completed:
Stored record for a run with manual LID assignments with docId = 
5b036573fa005d0e15f55894
Total Greened Acres = 3.9 with: RainBarrel = 3.9, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.55 sec
Run  40  completed:
Stored record for a run with manual LID assignments with docId = 
5b036576fa005d0e15f55895
Total Greened Acres = 4.0 with: RainBarrel = 4.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.73 sec
Run  41  completed:
Stored record for a run with manual LID assignments with docId = 
5b03657afa005d0e15f55896
Total Greened Acres = 4.1 with: RainBarrel = 4.1, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.58 sec
Run  42  completed:
Stored record for a run with manual LID assignments with docId = 
5b03657efa005d0e15f55897
Total Greened Acres = 4.2 with: RainBarrel = 4.2, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.90 sec
Run  43  completed:
Stored record for a run with manual LID assignments with docId = 
5b036580fa005d0e15f55898

0.0 min, 2.48 sec
Run  18  completed:
Stored record for a run with manual LID assignments with docId = 
5b0365e1fa005d0e15f558ba
Total Greened Acres = 1.8 with: RainBarrel = 1.8, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.49 sec
Run  19  completed:
Stored record for a run with manual LID assignments with docId = 
5b0365e4fa005d0e15f558bb
Total Greened Acres = 1.9 with: RainBarrel = 1.9, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.51 sec
Run  20  completed:
Stored record for a run with manual LID assignments with docId = 
5b0365e6fa005d0e15f558bc
Total Greened Acres = 2.0 with: RainBarrel = 2.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.49 sec
Run  21  completed:
Stored record for a run with manual LID assignments with docId = 
5b0365e9fa005d0e15f558bd
Total Greened Acres = 2.1 with: RainBarrel = 2.1, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.51 sec
Run  22  completed:
Stored record for a run with manual LID assignments with docId = 
5b0365ebfa005d0e15f558be

0.0 min, 2.92 sec
Run  56  completed:
Stored record for a run with manual LID assignments with docId = 
5b03664afa005d0e15f558e0
Total Greened Acres = 5.6 with: RainBarrel = 5.6, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.30 sec
Run  57  completed:
Stored record for a run with manual LID assignments with docId = 
5b03664efa005d0e15f558e1
Total Greened Acres = 5.7 with: RainBarrel = 5.7, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.61 sec
Run  58  completed:
Stored record for a run with manual LID assignments with docId = 
5b036651fa005d0e15f558e2
Total Greened Acres = 5.8 with: RainBarrel = 5.8, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.35 sec
Run  59  completed:
Stored record for a run with manual LID assignments with docId = 
5b036655fa005d0e15f558e3
Total Greened Acres = 5.9 with: RainBarrel = 5.9, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.34 sec
Run  60  completed:
Stored record for a run with manual LID assignments with docId = 
5b036658fa005d0e15f558e4

...

0.0 min, 2.96 sec
Run  18  completed:
Stored record for a run with manual LID assignments with docId = 
5b037305fa005d0e15f55d08
Total Greened Acres = 1.8 with: RainBarrel = 1.8, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.21 sec
Run  19  completed:
Stored record for a run with manual LID assignments with docId = 
5b037308fa005d0e15f55d09
Total Greened Acres = 1.9 with: RainBarrel = 1.9, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.98 sec
Run  20  completed:
Stored record for a run with manual LID assignments with docId = 
5b03730bfa005d0e15f55d0a
Total Greened Acres = 2.0 with: RainBarrel = 2.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.22 sec
Run  21  completed:
Stored record for a run with manual LID assignments with docId = 
5b03730efa005d0e15f55d0b
Total Greened Acres = 2.1 with: RainBarrel = 2.1, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.03 sec
Run  22  completed:
Stored record for a run with manual LID assignments with docId = 
5b037311fa005d0e15f55d0c

0.0 min, 3.03 sec
Run  7  completed:
Stored record for a run with manual LID assignments with docId = 
5b03737afa005d0e15f55d2e
Total Greened Acres = 0.7 with: RainBarrel = 0.7, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.89 sec
Run  8  completed:
Stored record for a run with manual LID assignments with docId = 
5b03737dfa005d0e15f55d2f
Total Greened Acres = 0.8 with: RainBarrel = 0.8, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.17 sec
Run  9  completed:
Stored record for a run with manual LID assignments with docId = 
5b037380fa005d0e15f55d30
Total Greened Acres = 0.9 with: RainBarrel = 0.9, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.98 sec
Run  10  completed:
Stored record for a run with manual LID assignments with docId = 
5b037383fa005d0e15f55d31
Total Greened Acres = 1.0 with: RainBarrel = 1.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.24 sec
Run  11  completed:
Stored record for a run with manual LID assignments with docId = 
5b037386fa005d0e15f55d32
To

0.0 min, 2.57 sec
Run  19  completed:
Stored record for a run with manual LID assignments with docId = 
5b0373e9fa005d0e15f55d54
Total Greened Acres = 1.9 with: RainBarrel = 1.9, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.37 sec
Run  20  completed:
Stored record for a run with manual LID assignments with docId = 
5b0373ebfa005d0e15f55d55
Total Greened Acres = 2.0 with: RainBarrel = 2.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.50 sec
Run  21  completed:
Stored record for a run with manual LID assignments with docId = 
5b0373eefa005d0e15f55d56
Total Greened Acres = 2.1 with: RainBarrel = 2.1, MorrisTT = 0.0, wakefield_BR_RG = 0.0,

In subshed  TT14-746_shed using LID  RainBarrel
0.0 min, 2.38 sec
Run  0  completed:
Stored record for a run with manual LID assignments with docId = 
5b0373f0fa005d0e15f55d57
Total Greened Acres = 0.0 with: RainBarrel = 0.0, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.41 sec
Run  1  completed:
Stored record for a run with manual LID ass

0.0 min, 2.33 sec
Run  16  completed:
Stored record for a run with manual LID assignments with docId = 
5b03744bfa005d0e15f55d7a
Total Greened Acres = 1.6 with: RainBarrel = 1.6, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.39 sec
Run  17  completed:
Stored record for a run with manual LID assignments with docId = 
5b03744efa005d0e15f55d7b
Total Greened Acres = 1.7 with: RainBarrel = 1.7, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.39 sec
Run  18  completed:
Stored record for a run with manual LID assignments with docId = 
5b037450fa005d0e15f55d7c
Total Greened Acres = 1.8 with: RainBarrel = 1.8, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.32 sec
Run  19  completed:
Stored record for a run with manual LID assignments with docId = 
5b037452fa005d0e15f55d7d
Total Greened Acres = 1.9 with: RainBarrel = 1.9, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.32 sec
Run  20  completed:
Stored record for a run with manual LID assignments with docId = 
5b037455fa005d0e15f55d7e

0.0 min, 2.75 sec
Run  14  completed:
Stored record for a run with manual LID assignments with docId = 
5b0374aafa005d0e15f55da0
Total Greened Acres = 1.4 with: RainBarrel = 1.4, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.41 sec
Run  15  completed:
Stored record for a run with manual LID assignments with docId = 
5b0374adfa005d0e15f55da1
Total Greened Acres = 1.5 with: RainBarrel = 1.5, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.45 sec
Run  16  completed:
Stored record for a run with manual LID assignments with docId = 
5b0374affa005d0e15f55da2
Total Greened Acres = 1.6 with: RainBarrel = 1.6, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 3.32 sec
Run  17  completed:
Stored record for a run with manual LID assignments with docId = 
5b0374b3fa005d0e15f55da3
Total Greened Acres = 1.7 with: RainBarrel = 1.7, MorrisTT = 0.0, wakefield_BR_RG = 0.0,
0.0 min, 2.86 sec
Run  18  completed:
Stored record for a run with manual LID assignments with docId = 
5b0374b5fa005d0e15f55da4